In [1]:
%pip install fastapi pydantic toml

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
from rs import skill_check

def test_skill_check(metadata):
    print(skill_check("sanity", None, metadata))

metadata = {
    "character": {"sanity":30},
    "symptom":[
        "incoherent speech",
        "uncontrollable twitching, trembling",
        "delusions of persecution",
        "strange appetites (dirt, clay, etc...)",
        "scratches, punches, and bruises all over the body"
    ]
}

test_skill_check(metadata)
test_skill_check(metadata)
test_skill_check(metadata)

print(metadata["character"])


I am making a SANITY check against my own sanity:30.
And... FAILED! I lost 2 sanity points, and gained a symptom: scratches, punches, and bruises all over the body.
I am making a SANITY check against my own sanity:28.
And... FAILED! I lost 4 sanity points, and gained a symptom: strange appetites (dirt, clay, etc...).
I am making a SANITY check against my own sanity:24.
And... FAILED! I lost 2 sanity points, and gained a symptom: delusions of persecution.
{'sanity': 22, 'symptoms': ['scratches, punches, and bruises all over the body', 'strange appetites (dirt, clay, etc...)', 'delusions of persecution']}


In [3]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_skillcheck_test.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You are standing in front of the haunted mansion. 
The mansion is old and decrepit, with ivy growing up the walls and the windows boarded up. 
The front door is slightly ajar, inviting you inside. 

Sunddenly you hear a loud crash from inside the mansion.

skill: sanity
difficulty: easy


I am making a SANITY check against my own sanity:22.
And... FAILED! I lost 2 sanity points, and gained a symptom: strange appetites (dirt, clay, etc...).
TURN: 1 


symptom: Your mind is clouded with an inexplicable craving for non-food items such as dirt, clay, and other such materials. This strange appetite gnaws at the edges of your consciousness, making it hard to focus on anything else.

You shake off the immediate effects of the crash sound, but the craving distracts you. Despite this, you push the front door open and step into the dark, musty interior of the mansion. The air is thick with dust, and the faint smell of decay lingers. You find yourself in a grand entrance hall, with a staircase leading up and several doors leading to other rooms on the ground floor.

possible actions:
1. Explore the room to the left.
2. Head towards the room to the right.
3. Investigate the sound upstairs.
4. Check out the room straight ahead.

I select option 2. TURN: 2 


You cautiously make your way towards the room to the right. As you approach, you notice that the door is slightly ajar. You push it open and enter what appears to be a library. The room is filled with shelves upon shelves of old books, their spines covered in dust. A large window on the far wall is covered with heavy curtains, blocking out most of the light, but you can see that it's still daytime outside.

In the center of the room, there's a large wooden desk with various papers and a strange, ancient-looking artifact sitting on top of it. The artifact is unlike anything you've seen before, and it seems to emit a faint, eerie glow.

possible actions:
1. Examine the artifact on the desk.
2. Search the shelves for any interesting books.
3. Look through the papers on the desk.
4. Leave the library and explore another room.

I select option 4. TURN: 3 


Leaving the library and its mysterious contents behind, you decide to explore another room. You exit back into the grand entrance hall and notice the door straight ahead seems to lead deeper into the mansion. You approach and open it, revealing a large dining room.

The dining room is grand, with a long table in the center surrounded by chairs. Dust covers everything, and cobwebs hang from the chandelier above. As you step into the room, a sudden movement catches your eye. A large, shadowy dog-like figure darts across the room and disappears through a door on the opposite side.

This unexpected sight startles you, potentially triggering a sanity check.

skill: sanity
difficulty: medium

I am making a SANITY check against my own sanity:20.
And... FAILED! I lost 1 sanity points, and gained a symptom: strange appetites (dirt, clay, etc...).
TURN: 4 


symptom: Your craving for non-food items intensifies, the desire for dirt, clay, and similar substances becoming almost overwhelming. It's becoming increasingly difficult to concentrate on your investigation as these urges cloud your thoughts.

Despite the unsettling encounter with the shadowy dog-like figure and the intensification of your strange appetites, you manage to maintain your composure. The door through which the creature disappeared stands ominously open, leading into darkness.

possible actions:
1. Follow the creature through the door.
2. Search the dining room for clues.
3. Return to the entrance hall and choose another room.
4. Try to find a kitchen or pantry, hoping to find something to address your strange cravings.